In [1]:
import torch
from torch import nn
from d2l import torch as d2l

互相关运算

In [2]:
def corr2d(x, k):
    h, w = k.shape
    y = torch.zeros((x.shape[0] - h + 1, x.shape[1] - w + 1))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (x[i:i + h, j: j + w] * k).sum() #在torch里面 * 就是互相关运算——即矩阵对应坐标的值相乘

    return y

卷积层

In [3]:
class Conv2D(nn.Module):

    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros([1,1]))

    def forward(self, x):
        y = corr2d(x, self.weight) + self.bias
        return y

简单的边缘检测

In [4]:
net = nn.Conv2d(1, 1, kernel_size = [1, 2], bias = False) #前两个1表示输入输出的通道

#形状都是高维度的写在前面的例如二维的[2,1],2是第二行,在张量里面数行就是1维的了,因为如果只有1维的话就代表只有1行,那么既然有2行那就说明是二维的
x = torch.ones([1, 1, 6, 8])
y = torch.zeros([1, 1, 6, 7])
#这里的形状是4维的,第一项代表batch也就是批量大小,在一个迭代中这个数值就代表这是这次迭代中的第几张图片
#第二项是通道数,普通的2D彩色图片一般有RGB 3个通道
#后面就是一个通道里的大小了
x[0, 0, :, 2:6] = 0
y[0, 0, :, 1] = 1
y[0, 0, :, 5] = -1
x, y

(tensor([[[[1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.],
           [1., 1., 0., 0., 0., 0., 1., 1.]]]]),
 tensor([[[[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
           [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]]]))

In [5]:
##现在要学一个w出来使他能做边缘检测
for i in range(10):
    y_hat = net(x)
    l = (y_hat - y) ** 2
    net.zero_grad() #使网络里所有的参数梯度清零
    l.sum().backward()
    net.weight.data -= 3e-2 * net.weight.grad

    print(f'batch {i+1}, loss: {l.sum()}')

batch 1, loss: 28.87433624267578
batch 2, loss: 12.239182472229004
batch 3, loss: 5.27701473236084
batch 4, loss: 2.3303258419036865
batch 5, loss: 1.0625720024108887
batch 6, loss: 0.5043945908546448
batch 7, loss: 0.25086554884910583
batch 8, loss: 0.13108451664447784
batch 9, loss: 0.07182344049215317
batch 10, loss: 0.041022807359695435


In [6]:
net.weight.data

tensor([[[[ 0.9641, -1.0012]]]])

填充和步长

In [7]:
padding1_net = nn.Conv2d(1, 1, kernel_size = 3, padding = 1) #kernel_size只写一个标量的话就默认是一个正方形

padding1_x = torch.rand([1, 1, 5, 5])

padding1_net(padding1_x).shape

torch.Size([1, 1, 5, 5])

In [8]:
#也可以做高度和宽带不同的填充
padding2_net = nn.Conv2d(1, 1, kernel_size = [5, 3], padding = [2, 1])

padding2_x = torch.rand([1, 1, 10, 10])

padding2_net(padding2_x).shape

torch.Size([1, 1, 10, 10])

In [9]:
#设置步长
stride_net = nn.Conv2d(1, 1, kernel_size = 3, padding = 1, stride = 2)

stride_x = torch.rand([1, 1, 5, 5])

stride_net(stride_x).shape

torch.Size([1, 1, 3, 3])

多输入通道互相关运算

In [10]:
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x ,k) for x, k in zip(X, K))
    #zip表示最大的一个维度,这就就表示对通道做遍历,这样拿出来的就是一个通道的x和k

多输出通道

In [14]:
def corr2d_multi_in_out(X, K):
    return torch.Stack(corr2d(X, k) for k in K) #stack可以把一个tensor堆叠,就扩充了维度,在这里就是把不同卷积核的输出stack一下就得到了输出